In [1]:
# library loading
import pandas as pd
import numpy as np
import re
import shutil
import os.path

In [2]:
# 매칭 키워드
## 스터디라인
tSL1 = ['기초이론', '기초서', '기본이론', '기본서', '입문이론', '입문서', '개념이론', '개념서', \
       '초보필독서', '기초필독서', '기본필독서', '입문필독서', '개념필독서']

tSL2 = ['OX','서브노트','두문자', '단원별기출문제', '단원별문제', '심화이론', '이론심화', '핵심이론',\
        '유형별문제집', '유형별핵심문제','유형별이론', '유형별기출문제', '유형별문제풀이']

tSL3 = ['0제', '빈출문제', '랜덤문제']

tSL4 = ['취약문제', '취약파트', '심화문제집', '심화문제', '고난이도문제', '고난도문제', '고난도이론',\
        '테마정리', '상위권문제']

tSL5 = ['실전모의고사', '봉투모의고사', '동형모의고사', '더블업', '더블업모의고사', '총정리', '최종',\
        '최종정리', '최종마무리', '필기노트']

## 책유형
title_t = ['입문서', '기초서', '기본서', '이론서', '개념서', '심화서', '요약서', '입문이론', '기초이론',\
           '기본이론', '개념이론', '핵심이론', '합격노트', '필기노트', '요약집']
etc_t = ['입문서', '기초서', '이론서', '개념서', '심화서', '요약서', '입문이론', '기초이론',\
         '기본이론', '개념이론', '핵심이론', '합격노트', '요약집', '이론정리', '기초정리',\
         '기본정리', '핵심정리', '요약정리', '이론요약',  '요약이론', '포인트이론',\
         '개념이해','개념학습']
title_q = ['문제집', '0제', '모의고사', '예상문제', '실전문제', '문제풀이','OX','기출문제','진도별']
etc_q = ['문제집', '0제', '모의고사', '예상문제', '실전문제', '문제풀이', '문제수록',\
           '빈출문제', '연도별기출', '핵심문제', '단원별문제', '심화문제', '심화문제집',\
           '실전문제', '실전모의고사', '동형모의고사', '더블업', '진도별모의고사', '핵심문제',\
           '봉투모의고사', '예상문제', '문제엄선']


## 과목
sub_ls = ['형법', '헌법', '형사법', '경찰행정학', '형사소송법', '경찰학', '경찰실무', '경찰행정법',\
       '주관식', '형사소송법', '경찰실무종합', '한국사','행정법총론', '행정학개론','사회복지학개론',\
       '영어', '세법개론', '정보보호론', '범죄학', '회계학', '시스템', '행정법', '행정학','사회복지학',\
       '민법총칙', '상법총칙', '경제학', '통계학', '재정학','네트워크보안', '데이터베이스론','voca',\
       '통신이론', '소프트웨어공학','화학공학일반','공업화학','교육학개론','교육학','국어','영문법']
## 급수
detail = ['9급','7/8급','7급','8급','6급','5급']


In [3]:
## function
def make_subject(sub, text):
    result = []
    for t in sub:
        if t in text:
            if t =='voca' or t=='영문법':
                t ='영어'
            
            result.append(t)
    
    if len(result) >= 1:
        result = result[0]
    else:
        result = ''
    return result
    
def make_type(title_t, title_q, text):
    result = ''
    lists = [['T' for tx in title_t if tx in text], # 이론서
             ['W' for tx in title_q if tx in text] # 문제풀이
                ]
    max_list = max(lists, key=len)
    if len(max_list) != 0:
        result = max_list[0]
    else:
        pass
    return result
    
def make_set(df, target, code):
    line, tex = [],[] # 위치, 과목(또는 타입) 정보 담는 리스트
    contents = '' # 해당위치에 텍스트 지정할 텍스트값
    for i in si[target]:
        tex.append(set_idv[code][i])
        line.append(i)
    # 과목 매칭 개수에 따른 값 지정
    tex = [t for t in tex if t != '']
    if len(tex) == len(line):
        contents = ''.join(set(tex))
    else:
        pass
    # 해당 위치에 과목 값 넣기
    df[code][line[0]-1] = contents

In [4]:
# data loading
data = pd.read_excel("C://info_data//new_book_08_08.xlsx") # 수집 원본 데이터

data['set_isbn'] = data['set_isbn'].astype(str)
data['isbn'] = data['isbn'].astype(str)

In [5]:
# 데이터 확인
data

,main_title,sub_title,cate,tot_page,author,publish,date,size,wei,img,...,ref_index,set_isbn,isbn,price,book_type,ver,last_ver,pn_isbn,keyword,cate_kb
0,2024 선재국어 수비니겨 기본서 1~2권 + 날로 쓰는 자습서 세트,"공단기, 공무원 시험 최신 경향과 신유형 전면 반영",공무원,760쪽,이선재,에스티유니타스,2023.07.15.,206*275*33mm,1690g,9791169660891.jpg,...,목차 교보문고,9791169660891,9791169660891,39000.0,종이책 - 무선제본,NaN,NaN,NaN,NaN,"['국내도서', '공무원 국어', '취업/수험서', '공무원 과목별']"
1,2024 선재국어 수비니겨 기본서 1 독해 · 문학,NaN,공무원,NaN,이선재,에스티유니타스,NaN,NaN,NaN,NaN,...,NaN,9791169660891,9791169660907,NaN,종이책 - 무선제본,11판,9.791166e+12,256,공무원;국어;기본서,NaN
2,2024 선재국어 수비니겨 날로 쓰는 자습서,NaN,공무원,NaN,이선재,에스티유니타스,NaN,NaN,NaN,NaN,...,NaN,9791169660891,9791169660921,NaN,종이책 - 무선제본,11판,NaN,184,공무원;국어;기본서,NaN
3,2024 선재국어 수비니겨 기본서 2 국어 문법 · 국어 규범,NaN,공무원,NaN,이선재,에스티유니타스,NaN,NaN,NaN,NaN,...,NaN,9791169660891,9791169660914,NaN,종이책 - 무선제본,11판,9.791166e+12,320,공무원;국어;기본서,NaN
4,2024 이동기 영어 문법·구문 50포인트,이동기 영어 기본서 (문법·구문),공무원,480쪽,이동기,에스티유니타스,2023.05.12.,207*263*20mm,1054g,9791169660549.jpg,...,목차 알라딘에,0,9791169660549,14000.0,종이책 - 무선제본,NaN,NaN,NaN,NaN,"['국내도서', '공무원 영어', '취업/수험서', '공무원 과목별']"
5,2024 이동기 영어 독해 12공식,공무원 영어의 시작과 끝,공무원,304쪽,이동기,에스티유니타스,2023.07.03.,206*262*13mm,724g,9791169660556.jpg,...,목차 교보문고,0,9791169660556,20000.0,종이책 - 무선제본,NaN,NaN,NaN,NaN,"['국내도서', '공무원 영어', '취업/수험서', '공무원 과목별']"
6,2024 이동기 공무원 마스터 VOCA 2000+,심화 어휘 200 생활 영어,공무원,512쪽,이동기,지금,2023.07.07.,156*224*23mm,733g,9791160183658.jpg,...,목차 교보문고,0,9791160183658,24000.0,종이책 - 무선제본,10판,9.791160e+12,NaN,공무원영어;이동기영어;공무원어휘;공무원보카,"['국내도서', '공무원 영어', '취업/수험서', '공무원 과목별']"
7,2024 문동균 한국사 All in one,NaN,공무원,928쪽,문동균,에스티유니타스,2023.05.17.,205*259*37mm,1840g,9791169660419.jpg,...,목차 교보문고,9791169660419,9791169660419,45000.0,종이책 - 무선제본,NaN,NaN,NaN,NaN,"['국내도서', '공무원 한국사', '취업/수험서', '공무원 과목별']"
8,문동균 한국사 all in one 2 - 공무원 한국사,NaN,공무원,NaN,문동균,에스티유니타스,NaN,NaN,NaN,NaN,...,NaN,9791169660419,9791169660433,22500.0,종이책 - 무선제본,"8판, 2024 개정판",9.791166e+12,598-928,한국사; 공무원한국사;,NaN
9,문동균 한국사 all in one 1 - 공무원 한국사,NaN,공무원,NaN,문동균,에스티유니타스,NaN,NaN,NaN,NaN,...,NaN,9791169660419,9791169660426,22500.0,종이책 - 무선제본,"8판, 2024 개정판",9.791166e+12,588,한국사; 공무원한국사;,NaN


In [6]:
codes = data['set_isbn']
code = data['isbn']
tit = data['main_title']
rev = data['book_review']
info = data['book_info']

In [7]:
# 과목, 유형 작업

isbn_ls, set_ls, title_ls = [],[],[]
subj_ls, type_ls = [],[]

for i in range(len(tit)):
    sub = ''
    typ = ''
    if code[i] == codes[i]:
        pass
    else:
        text = tit[i].replace(' ','')
        text= text.lower()
        sub = make_subject(sub_ls, text)
        typ = make_type(title_t, title_q, text)
    
    isbn_ls.append(code[i])
    set_ls.append(codes[i])
    title_ls.append(tit[i])
    subj_ls.append(sub)
    type_ls.append(typ)
    
df = pd.DataFrame({'제목':title_ls,'isbn':isbn_ls, 'set_isbn':set_ls,'subj':subj_ls, 'type': type_ls})
df

,제목,isbn,set_isbn,subj,type
0,2024 선재국어 수비니겨 기본서 1~2권 + 날로 쓰는 자습서 세트,9791169660891,9791169660891,,
1,2024 선재국어 수비니겨 기본서 1 독해 · 문학,9791169660907,9791169660891,국어,T
2,2024 선재국어 수비니겨 날로 쓰는 자습서,9791169660921,9791169660891,국어,
3,2024 선재국어 수비니겨 기본서 2 국어 문법 · 국어 규범,9791169660914,9791169660891,국어,T
4,2024 이동기 영어 문법·구문 50포인트,9791169660549,0,영어,
5,2024 이동기 영어 독해 12공식,9791169660556,0,영어,
6,2024 이동기 공무원 마스터 VOCA 2000+,9791160183658,0,영어,
7,2024 문동균 한국사 All in one,9791169660419,9791169660419,,
8,문동균 한국사 all in one 2 - 공무원 한국사,9791169660433,9791169660419,한국사,
9,문동균 한국사 all in one 1 - 공무원 한국사,9791169660426,9791169660419,한국사,


In [8]:
# 세트 개별권의 인덱스와 과목 정보 확인

## 필요 데이터프레임 정리
set_idv = df.loc[(df['set_isbn'] != df['isbn']) & (df['set_isbn']!='0'), ('set_isbn','isbn','subj','type')]
## 세트_isbn : 인덱스
si = set_idv.groupby('set_isbn').groups
target_set = list(si.keys()) # 세트_isbn 리스트화

In [9]:
# 세트권 과목 작업 
for ts in target_set:
    make_set(df, ts, 'subj')
    make_set(df, ts, 'type')

In [10]:
df

,제목,isbn,set_isbn,subj,type
0,2024 선재국어 수비니겨 기본서 1~2권 + 날로 쓰는 자습서 세트,9791169660891,9791169660891,국어,
1,2024 선재국어 수비니겨 기본서 1 독해 · 문학,9791169660907,9791169660891,국어,T
2,2024 선재국어 수비니겨 날로 쓰는 자습서,9791169660921,9791169660891,국어,
3,2024 선재국어 수비니겨 기본서 2 국어 문법 · 국어 규범,9791169660914,9791169660891,국어,T
4,2024 이동기 영어 문법·구문 50포인트,9791169660549,0,영어,
5,2024 이동기 영어 독해 12공식,9791169660556,0,영어,
6,2024 이동기 공무원 마스터 VOCA 2000+,9791160183658,0,영어,
7,2024 문동균 한국사 All in one,9791169660419,9791169660419,한국사,
8,문동균 한국사 all in one 2 - 공무원 한국사,9791169660433,9791169660419,한국사,
9,문동균 한국사 all in one 1 - 공무원 한국사,9791169660426,9791169660419,한국사,


In [17]:
len(df.loc[df['type'] == ''])


43

In [117]:
df['SL'] = ''
df

,제목,isbn,set_isbn,subj,type,SL
0,2024 선재국어 수비니겨 기본서 1~2권 + 날로 쓰는 자습서 세트,9791169660891,9791169660891,국어,,
1,2024 선재국어 수비니겨 기본서 1 독해 · 문학,9791169660907,9791169660891,국어,T,
2,2024 선재국어 수비니겨 날로 쓰는 자습서,9791169660921,9791169660891,국어,,
3,2024 선재국어 수비니겨 기본서 2 국어 문법 · 국어 규범,9791169660914,9791169660891,국어,T,
4,2024 이동기 영어 문법·구문 50포인트,9791169660549,0,영어,,
5,2024 이동기 영어 독해 12공식,9791169660556,0,영어,,
6,2024 이동기 공무원 마스터 VOCA 2000+,9791160183658,0,영어,,
7,2024 문동균 한국사 All in one,9791169660419,9791169660419,한국사,,
8,문동균 한국사 all in one 2 - 공무원 한국사,9791169660433,9791169660419,한국사,,
9,문동균 한국사 all in one 1 - 공무원 한국사,9791169660426,9791169660419,한국사,,


# 과목 데이터 입력

In [ ]:
# 필요한 데이터 로딩(과목코드)
subjCD = pd.read_excel("C:\info_data\subj_cd_09_11.xlsx")

In [126]:
# 과목 테이블에 필요한 부분만 로딩
txtb_sor = df.loc[df['subj'] != '', ('isbn', 'subj')]

# 교재아이디
txtb_sor['isbn'] = txtb_sor['isbn'].apply(lambda x: f'm{x}')

In [129]:
# 과목 - 코드 매칭 
txtb_sor = pd.merge(txtb_sor, subjCD, left_on = 'subj', right_on = 'subj_nm')
txtb_sor = txtb_sor[['isbn','subj_cd']] # 필요한 컬럼만 남기기
txtb_sor.columns = ['txtb_id', 'subj_cd'] # 과목코드명 변경

In [131]:
# 데이터 저장
txtb_sor.to_csv("저장할 위치", encoding = 'utf-8-sig', index = False)

,txtb_id,subj_cd
0,m9791169660891,ko
1,m9791169660907,ko
2,m9791169660921,ko
3,m9791169660914,ko
4,m9791138328234,ko
5,m9791165769277,ko
6,m9791192083483,ko
7,m9791169660549,en
8,m9791169660556,en
9,m9791160183658,en


# 수작업 파일화

In [133]:
hand = df[['제목','isbn','subj','type','SL']]
hand

,제목,isbn,subj,type,SL
0,2024 선재국어 수비니겨 기본서 1~2권 + 날로 쓰는 자습서 세트,9791169660891,국어,,
1,2024 선재국어 수비니겨 기본서 1 독해 · 문학,9791169660907,국어,T,
2,2024 선재국어 수비니겨 날로 쓰는 자습서,9791169660921,국어,,
3,2024 선재국어 수비니겨 기본서 2 국어 문법 · 국어 규범,9791169660914,국어,T,
4,2024 이동기 영어 문법·구문 50포인트,9791169660549,영어,,
5,2024 이동기 영어 독해 12공식,9791169660556,영어,,
6,2024 이동기 공무원 마스터 VOCA 2000+,9791160183658,영어,,
7,2024 문동균 한국사 All in one,9791169660419,한국사,,
8,문동균 한국사 all in one 2 - 공무원 한국사,9791169660433,한국사,,
9,문동균 한국사 all in one 1 - 공무원 한국사,9791169660426,한국사,,


In [ ]:
hand.to_excel("저장경로", encoding='utf-8-sig',index = False)